## 🧑‍🎓 Explaining the Parametric Sphere and Manim's `Surface` Class

![](./assets/sphcoordel.png)

---

### 1. The Parametric Sphere Function (`sphere_func`)

To draw a 3D object like a sphere without using a built-in Mobject, we must define its surface using **parametric equations**. These equations use two independent variables (parameters, usually $u$ and $v$ in Manim) to determine the $x$, $y$, and $z$ coordinates of every point on the surface.

The mathematical basis for the sphere's function is **Spherical Coordinates** $(\rho, \phi, \theta)$, where:
* $\rho$ (rho) is the distance from the origin (your `radius`).
* $\phi$ (phi, or Manim's $u$) is the **polar angle** (down from the positive Z-axis).
* $\theta$ (theta, or Manim's $v$) is the **azimuthal angle** (around the Z-axis).

The conversion formulas to Cartesian coordinates ($x, y, z$) are:

$$
\begin{align*}
x &= \rho \cdot \sin(\phi) \cdot \cos(\theta) \\
y &= \rho \cdot \sin(\phi) \cdot \sin(\theta) \\
z &= \rho \cdot \cos(\phi)
\end{align*}
$$

#### ⚙️ How it translates to the Manim function:

| Manim Variable | Math Symbol | Name | Range | Purpose |
| :---: | :---: | :---: | :---: | :--- |
| `u` | $\phi$ | Polar Angle | $0$ to $\pi$ | Moves from the **top pole** (Z-axis) to the **bottom pole** ($\pi$). |
| `v` | $\theta$ | Azimuthal Angle | $0$ to $2\pi$ | Spins the point **360 degrees** around the Z-axis. |
| `radius` | $\rho$ | Radius | (Constant) | The size of the sphere. |

By using the correct ranges for $u$ and $v$, the function covers the **entire** spherical surface exactly once.

---

### 2. The `Surface` Class in Manim

The **`Surface`** Mobject is Manim's primary tool for rendering continuous 3D shapes defined by parametric equations. It takes your function and translates the continuous range of $u$ and $v$ into a 3D object composed of small geometric patches.

| Parameter | Purpose | Your Value |
| :--- | :--- | :--- |
| `func` | **Required.** The Python function (like `sphere_func`) that takes two numeric arguments ($u$ and $v$) and returns a 3-element `numpy.array` ($[x, y, z]$). | `sphere_func` |
| `u_range` | Defines the start and end values for the first parameter ($u$ or $\phi$). | `[0, np.pi]` |
| `v_range` | Defines the start and end values for the second parameter ($v$ or $\theta$). | `[0, 2 * np.pi]` |
| `resolution` | **Crucial for quality.** A tuple `(u_res, v_res)` that sets how many points are calculated along the $u$ and $v$ directions. Higher values (like `(64, 64)`) create a smoother surface. | `(64, 64)` |

The animation **`Create(parametric_sphere)`** then draws these calculated geometric patches onto the screen sequentially, creating the effect of the sphere being built from its mathematical definition.

In [1]:
from manim import *
import numpy as np

class CompleteParametricSphereExplained(ThreeDScene):
    def construct(self):
        # 1. Initial Camera Setup and Axes
        self.set_camera_orientation(
            phi=70 * DEGREES, 
            theta=-45 * DEGREES, 
            distance=7
        )
        
        # --- Explanation Text Mobjects (Positions Adjusted) ---
        
        title = Text("Sphere as a Parametric Surface", font_size=40).to_edge(UP)
        
        # Note 1: Parametric Function Setup - Shifted slightly down from UP
        note_func = VGroup(
            Text("1. Define Parametric Function (x, y, z):", font_size=28, color=YELLOW),
            # FIX: Using MathTex for equations
            MathTex(r"x = R \sin(\phi) \cos(\theta)", font_size=30),
            MathTex(r"y = R \sin(\phi) \sin(\theta)", font_size=30),
            MathTex(r"z = R \cos(\phi)", font_size=30),
            # Using Text for the plain text range description
            MathTex(r"\text{Range: } \phi \in [0, \pi], \theta \in [0, 2\pi] \text{, defines range for u and v}", font_size=28),
            Text("R is the sphere of radius", font_size=20, color=WHITE),
        ).arrange(DOWN, aligned_edge=LEFT).shift(UP * 1.5 + DOWN * 0.5) # Adjust Y-position to be in the middle-top
        
        # Note 2: Surface Creation - Shifted to the middle of the screen
        note_surface = VGroup(
            Text("2. Create Manim Surface:", font_size=28, color=YELLOW),
            Text("Mobject: Surface(func, u_range, v_range, resolution)", font_size=24, color=WHITE)
        ).arrange(DOWN, aligned_edge=LEFT).shift(DOWN * 1.5 + RIGHT * 0.8) # Adjust Y-position to be in the middle

        # Fix the text to the camera
        self.add_fixed_in_frame_mobjects(title, note_func, note_surface)
        
        # --- PHASE 1: Display and Fade Out Explanation ---
        
        # Step 1: Show all explanation text
        self.play(
            Write(title), 
            Write(note_func),
            Write(note_surface)
        )
        self.wait(3)

        # Step 2: Remove all text Mobjects (must remove from fixed mobjects list first)
        self.remove_fixed_in_frame_mobjects(title, note_func, note_surface)
        self.play(
            FadeOut(note_func), 
            FadeOut(note_surface),
            run_time=1.5
        )
        self.wait(0.2)
        
        # --- PHASE 2: Sphere Creation and Rotation ---

        axes = ThreeDAxes()
        self.add(axes)

        # 2. Define the Sphere Parametric Function (for the Mobject)
        radius = 1.5
        def sphere_func(u, v):
            x = radius * np.sin(u) * np.cos(v)
            y = radius * np.sin(u) * np.sin(v)
            z = radius * np.cos(u)
            return np.array([x, y, z])

        # # 3. Create the Surface Mobject
        parametric_sphere = Surface(
            func=sphere_func,
            u_range=[0, np.pi],
            v_range=[0, 2 * np.pi],
            resolution=(64, 64), 
            color=TEAL,
            fill_opacity=0.8,
            checkerboard_colors=[TEAL, BLUE_D]
        )
        
        # # Step 3: Draw the sphere
        self.play(
            Create(parametric_sphere),
            run_time=4
        )
        self.wait(1)

        # # Step 4: Rotate the camera around the fully drawn sphere
        self.begin_ambient_camera_rotation(rate=0.2, about='theta')

        # # Let the animation run
        self.wait(10)

        # # Stop rotation and clean up
        self.stop_ambient_camera_rotation()
        self.wait(1)

%manim -ql -v warning CompleteParametricSphereExplained

Manim Community v0.19.0